### Table of Content
- [Import Data](#Import-Data)
- [Inspect Data](#Inspect-Data)
- [Clean Data](#Clean-Data)

In [ ]:
# import packages
import pandas as pd
import numpy as np
from pathlib import Path

### Import Data
---------------------------

In this workout, we use a dataset of [cafe sales](https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training) from kaggle to show how to use data cleaning techniques to process dirty data.

In [10]:
# Load the café sales dataset
data_path = Path("data/dirty_cafe_sales.csv")

# Check if the file exists
if data_path.exists():
    df = pd.read_csv(data_path)
    print(f"Data loaded successfully. {df.shape[0]} rows and {df.shape[1]} columns.")
else:
    raise FileNotFoundError(f"The file {data_path} does not exist.")

Data loaded successfully. 10000 rows and 8 columns.


### Inspect Data
----------------

In [11]:
# Take a first look
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [12]:
print("\nDataFrame Information:")
df.info()

print("\nMissing values in each column:")
df.isnull().sum()


DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB

Missing values in each column:


Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

Through data inspecting we found that:
- column names are not consistent: the names contain white space, lowercase and capital letters.
- the data type of all columns are not correct, because of the messy data values, pandas cannot correctly recognize the data type.
- some columns have wrong values, like `ERROR` and `UNKNOWN`
- some columns have null values. 

So next we will dealing with these problems through data cleaning. 

### Clean Data
--------------

#### Standardize column names

In order to keep the consistency of column names between various tools and database, it's a good practice to remove the whitespace at the beginning and end of the column names, keep all names lowercase and replace the whitespace with underscore `_` in the column names. 

**Objectives**:
- Remove spaces at beginning and end of column names
- Lowercase column names 
- Replace white space between names with `_`


**Methods**: `str.strip()`, `str.lower()`, `str.replace()`.


In [13]:
# --- 1. Standardize column names ---
# Standardize column names: strip whitespace, convert to lowercase, and replace spaces with underscores

df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_', regex=False)
) 

print("\nStandardizing column names:")
df.columns



Standardizing column names:


Index(['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent',
       'payment_method', 'location', 'transaction_date'],
      dtype='object')

#### Covert column types

Through the first check of the data sample and the metadata of of the cafe sales dataset, we know that the data type of each column should be like:
| Column           | Data Type category| 
|------------------|-------------------|
| transaction_id   | text              | 
| item             | text              | 
| quantity         | number            | 
| price_per_unit   | number            |
| total_spent      | number            |
| payment_method   | text              |
| location         | text              |
| transaction_date | date              |

So next step we convert the data type of each column.

### Convert numerical columns

**Objectives**:
- Remove commas and spaces in the data values
- Convert data values to numbers 
- Replace invalid values with `NaN`.

**Methods**: `replace()`, `pd.to_numeric()`.

In [ ]:
# --- 2. Convert numerical columns ---
num_cols = ['quantity', 'price_per_unit', 'total_spent']
df[num_cols] = (
    df[num_cols]
    .replace(r'[, \s]', '', regex=True)  # Remove commas and spaces
    .apply(pd.to_numeric, errors='coerce') # Convert to numeric, coercing errors to NaN
)

### Convert date column

**Objectives**:
- Convert transaction date strings into datetime objects.
- Convert invalid formats to NaN.

**Methods**: `replace()`, `pd.to_datetime()`.

In [15]:
# --- 3. Convert date columns ---
date_cols = ['transaction_date']

## Convert all non-string values to empty string, then replace invalid values with np.nan
df[date_cols] = df[date_cols].map(lambda x: str(x).strip() if pd.notnull(x) else '')
df[date_cols] = df[date_cols].replace({'': np.nan, 'unknown': np.nan, 'nan': np.nan}, regex=False)

# Convert to datetime, let pandas infer the format, coerce errors to NaT
df[date_cols[0]] = pd.to_datetime(df[date_cols[0]], errors='coerce')

### Convert categorical string columns

**Objectives**:
- Remove whitespace 
- Lowercase strings
- Replace invalid strings with `NaN`

**Methods**: `str.strip()`, `str.lower()`, `replace()`.

In [16]:
# --- 4. Convert text columns ---
text_cols = ['transaction_id', 'item', 'payment_method', 'location']
df[text_cols] = df[text_cols].apply(
    lambda x: (x.str.strip()   # Strip whitespace
               .str.lower()  # Convert to lowercase
               .replace({'':np.nan, 'unknown':np.nan, 'nan':np.nan}, regex=False)) # Replace empty strings and 'unknown' with NaN
)

### Check data types after conversion

In [ ]:
# Check the data types of the numeric columns
print("\nData types of columns after conversion:")
df.dtypes


Data types of columns after conversion:


transaction_id              object
item                        object
quantity                   float64
price_per_unit             float64
total_spent                float64
payment_method              object
location                    object
transaction_date    datetime64[ns]
dtype: object

In [18]:
df.head()

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,txn_1961373,coffee,2.0,2.0,4.0,credit card,takeaway,2023-09-08
1,txn_4977031,cake,4.0,3.0,12.0,cash,in-store,2023-05-16
2,txn_4271903,cookie,4.0,1.0,NaN,credit card,in-store,2023-07-19
3,txn_7034554,salad,2.0,5.0,10.0,NaN,NaN,2023-04-27
4,txn_3160411,coffee,2.0,2.0,4.0,digital wallet,in-store,2023-06-11
